In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install python-dotenv
# import kagglehub
# import os
# from dotenv import load_dotenv

# # Zdefiniuj ścieżkę do pliku .env na Twoim Google Drive
# dotenv_path = '/content/drive/MyDrive/API_KEY/kaggle.env'

# # Sprawdź, czy plik istnieje przed próbą jego wczytania
# if os.path.exists(dotenv_path):
#     load_dotenv(dotenv_path)

#     # Pobierz zmienne środowiskowe
#     kaggle_login = os.getenv('KAGGLE_LOGIN')
#     kaggle_key = os.getenv('KAGGLE_KEY')

#     # # Możesz teraz użyć kaggle_login i kaggle_key
#     # print(f"KAGGLE_LOGIN: {kaggle_login}")
#     # print(f"KAGGLE_KEY: {kaggle_key}")
# else:
#     print(f"Plik .env nie został znaleziony w {dotenv_path}")
# os.environ['KAGGLE_USERNAME'] = kaggle_login
# os.environ['KAGGLE_KEY'] = kaggle_key

# kagglehub.login()


In [ ]:
# print('Importing data sources...')

# competition_path = kagglehub.competition_download('digit-recognizer')

# print('Data source import complete.')

In [ ]:
# # List files in the downloaded directory
# print(f"Files in the downloaded directory ({competition_path}):")
# for dirname, _, filenames in os.walk(competition_path):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# Wczytanie danych
# ds_train = pd.read_csv(os.path.join(competition_path, "train.csv"))
# ds_test = pd.read_csv(os.path.join(competition_path, "test.csv"))
ds_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
ds_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(f'Train shape: {ds_train.shape}')
print(f'Test shape: {ds_test.shape}')

Train shape: (42000, 785)
Test shape: (28000, 784)


In [20]:
from sklearn.model_selection import train_test_split

X_train_full = ds_train.drop('label', axis=1)
y_train_full = ds_train['label']
y_train = y_train_full

# Normalizacja pikseli

X_train = X_train_full / 255
X_test = ds_test / 255

# # Podział na zbiór treningowy i walidacyjny
# X_train, X_valid = X_train_full[:-2000], X_train_full[-2000:]
# y_train, y_valid = y_train_full[:-2000], y_train_full[-2000:]

In [21]:
# Ustaw ziarno losowości
import random

SEED = 42
# os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [22]:
# Dodanie wymiaru kanału dla "Splotowej Sieci Neuronowej"
X_train = X_train.values.reshape(-1, 28, 28, 1)
# X_valid = X_valid.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)


In [23]:
y_train = y_train.astype('int')

In [24]:
X=X_train
y=y_train

In [25]:
# Model
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.BatchNormalization(),  # dodaj batch norm
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.3),
    
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.3),
    
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# === 3. Konfiguracja augmentacji ===
# Tylko dla danych treningowych — nie dla walidacyjnych!
datagen = ImageDataGenerator(
    rotation_range=10,        # losowy obrót w zakresie [-10°, 10°]
    zoom_range=0.1,           # przybliżenie losowo w zakresie [0.9, 1.1]
    width_shift_range=0.1,    # przesunięcie poziome (maks. 10%)
    height_shift_range=0.1    # przesunięcie pionowe (maks. 10%)
)

**Stratified Cross Validation**


In [29]:
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []
models_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n📂 Fold {fold + 1}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    model = build_model()
    
    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=5,             # zatrzymaj po 5 epokach bez poprawy
        restore_best_weights=True
    )

    datagen.fit(X_train)

    #Trening
    model.fit(
        datagen.flow(X_train, y_train, batch_size=64),  # dane augmentowane w locie
        validation_data=(X_val, y_val),                 # bez augmentacji
        epochs=30,
        callbacks=[early_stop],
        verbose=1
    )

    models_list.append(model)  # zapisujemy model z tego folda
    # Ocena
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"✅ Fold {fold+1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

# Średnia dokładność
mean_acc = np.mean(fold_accuracies)
print(f"\n📊 Średnia dokładność z {skf.n_splits} foldów: {mean_acc:.4f}")


📂 Fold 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.5810 - loss: 1.2802 - val_accuracy: 0.9660 - val_loss: 0.1275
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9180 - loss: 0.2681 - val_accuracy: 0.9796 - val_loss: 0.0654
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9473 - loss: 0.1832 - val_accuracy: 0.9881 - val_loss: 0.0472
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9605 - loss: 0.1336 - val_accuracy: 0.9899 - val_loss: 0.0389
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9687 - loss: 0.1075 - val_accuracy: 0.9883 - val_loss: 0.0421
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9686 - loss: 0.1039 - val_accuracy: 0.9898 - val_loss: 0.0377
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9720 - loss: 0.0990 - val_accuracy: 0.9904 - val_loss: 0.0375
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9725 - loss: 0.0897 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.5905 - loss: 1.2505 - val_accuracy: 0.9707 - val_loss: 0.1152
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9165 - loss: 0.2738 - val_accuracy: 0.9806 - val_loss: 0.0595
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9466 - loss: 0.1801 - val_accuracy: 0.9843 - val_loss: 0.0477
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9573 - loss: 0.1455 - val_accuracy: 0.9871 - val_loss: 0.0417
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9666 - loss: 0.1135 - val_accuracy: 0.9895 - val_loss: 0.0304
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9676 - loss: 0.1128 - val_accuracy: 0.9893 - val_loss: 0.0347
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9718 - loss: 0.1006 - val_accuracy: 0.9896 - val_loss: 0.0317
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9744 - loss: 0.0868 - 

...

Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - accuracy: 0.5754 - loss: 1.2915 - val_accuracy: 0.8768 - val_loss: 0.4149
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9205 - loss: 0.2579 - val_accuracy: 0.9860 - val_loss: 0.0459
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9490 - loss: 0.1740 - val_accuracy: 0.9867 - val_loss: 0.0428
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9605 - loss: 0.1380 - val_accuracy: 0.9892 - val_loss: 0.0393
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9672 - loss: 0.1161 - val_accuracy: 0.9896 - val_loss: 0.0424
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9723 - loss: 0.0964 - val_accuracy: 0.9886 - val_loss: 0.0389
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9733 - loss: 0.0945 - val_accuracy: 0.9888 - val_loss: 0.0401
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9752 - loss: 0.0859 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - accuracy: 0.5997 - loss: 1.2532 - val_accuracy: 0.9338 - val_loss: 0.2467
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9224 - loss: 0.2491 - val_accuracy: 0.9810 - val_loss: 0.0609
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9499 - loss: 0.1726 - val_accuracy: 0.9862 - val_loss: 0.0426
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9573 - loss: 0.1445 - val_accuracy: 0.9906 - val_loss: 0.0323
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9627 - loss: 0.1211 - val_accuracy: 0.9906 - val_loss: 0.0296
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9703 - loss: 0.1057 - val_accuracy: 0.9917 - val_loss: 0.0272
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9717 - loss: 0.0994 - val_accuracy: 0.9921 - val_loss: 0.0275
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9752 - loss: 0.0881 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.6177 - loss: 1.1686 - val_accuracy: 0.9611 - val_loss: 0.1276
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9300 - loss: 0.2320 - val_accuracy: 0.9858 - val_loss: 0.0552
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9523 - loss: 0.1631 - val_accuracy: 0.9856 - val_loss: 0.0508
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9643 - loss: 0.1264 - val_accuracy: 0.9888 - val_loss: 0.0415
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9659 - loss: 0.1175 - val_accuracy: 0.9856 - val_loss: 0.0504
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9709 - loss: 0.1012 - val_accuracy: 0.9863 - val_loss: 0.0455
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9738 - loss: 0.0953 - val_accuracy: 0.9910 - val_loss: 0.0336
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9759 - loss: 0.0874 - 

In [30]:
# === 6. Ensemble predykcji ===
print("\n📊 Ensemble predykcji z 5 modeli...")
all_preds = np.zeros((X_test.shape[0], 10))

for model in models_list:
    pred = model.predict(X_test, verbose=0)
    all_preds += pred

ensemble_preds = np.argmax(all_preds / len(models_list), axis=1)


📊 Ensemble predykcji z 5 modeli...


In [ ]:
# history = model.fit(X_train, y_train,
#                     batch_size=64,
#                     epochs=30,
#                     validation_data=(X_valid, y_valid),
#                     shuffle=False,
#                     callbacks=[early_stop])


# def plot_history(history):
#     plt.figure(figsize=(12, 4))

#     # Accuracy
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['accuracy'], label='train acc')
#     if 'val_accuracy' in history.history:
#         plt.plot(history.history['val_accuracy'], label='val acc')
#     plt.title('Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.legend()

#     # Loss
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['loss'], label='train loss')
#     if 'val_loss' in history.history:
#         plt.plot(history.history['val_loss'], label='val loss')
#     else:
#         print("⚠️ Nie znaleziono 'val_loss' — czy podałeś validation_data?")
#     plt.title('Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

# plot_history(history)

In [17]:
# y_pred_proba = model.predict(X_test)
# y_pred = np.argmax(y_pred_proba, axis=1)

875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [33]:
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(ensemble_preds) + 1),
    "Label": ensemble_preds
})

submission.to_csv("07_submission.csv", index=False)